<a href="https://colab.research.google.com/github/kcelestinomaria/qcHackChallenge/blob/main/QuantumArtificialNeuralNetWithQiskitPulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Our Work below, our intention was to implement a Quantum Artificial Neural Network, and then use Qiskit Pulse in order to control noise and reduce errors. Previous work has shown Quantum Artificial Neural Networks as a possible application of quantum circuits in AI. However, there has not been any noticed advantage when Quantum Artificial Neural Networks have been implemented and ran side by side to Classical Artificial Neural Networks. This has largely been because of the noise that affects the qubits from the environment. We therefore wanted to use Qiskit Pulse, and by accessing the low-level part of the quantum device through the IBM Cloud, we could then reduce noise and then test to see if there would be an effect of the performance of the Quantum Artificial Neural Network

We implemented a simple quantum-classical neural network to find any advantage over a simple neural network

First, let's import the needed libraries

In [ ]:
import torch

from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

import qiskit
from qiskit.tools.jupyter import *
from qiskit import *
 from qiskit.scheduler import measure

from qiskit.visualization import *
from qiskit import pulse            # This is where we access all of our Pulse features!
from qiskit.pulse import Play
# This Pulse module helps us build sampled pulses for common pulse shapes
from qiskit.pulse import library as pulse_lib

We then implement Quantum functions using qiskit modularized in a class. To keep it simple, we used a 1-qubit circuit with one trainable quantum parameter

In [ ]:
class QuantumCircuit:
    """ 
   To capitalize on Python's OOP features, we use this as a simple way to interface or rather represent the quantum function
    """
    
    def __init__(self, n_qubits, backend, shots):
        # Circuit initialization and definition
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        
        self._circuit.measure_all()
       
        self.backend = backend
        self.shots = shots
    
    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        result = job.result().get_counts(self._circuit)
        
        counts = np.array(list(result.values()))
        states = np.array(list(result.keys())).astype(float)
        
        # Computing probabilities for each state
        probabilities = counts / self.shots
        # Get state expectation
        expectation = np.sum(states * probabilities)
        
        return np.array([expectation])

We then test the above implementation of the quantum circuit against a real quantum computer on the IBM Cloud

In [ ]:
simulator = qiskit.Aer.get_backend('qasm_simulator')

circuit = QuantumCircuit(1, simulator, 100)
print('Expected value for rotation pi {}'.format(circuit.run([np.pi])[0]))
circuit._circuit.draw()

We will also implement a hybrid quantum-classical function

In [ ]:
class HybridFunction(Function):
    """ Defining a hybrid quantum-classical funtion """
    
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward propagation computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        expectation_z = ctx.quantum_circuit.run(input[0].tolist())
        result = torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)

        return result
        
    @staticmethod
    def back(ctx, grad_output):
        """ Backward propagation computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None

class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(1, backend, shots)
        self.shift = shift
        
    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit, self.shift)

In order to now run our neural networks, we need to import and process some data

As such, We created a simple hybrid neural network to classify images of two types of digits (0 or 1) from the Deep Learning practitioner Yann Lecun's data database [MNIST dataset](http://yann.lecun.com/exdb/mnist/). We first load MNIST and filter for pictures containing 1's and 0's. These will serve as inputs for our neural network to classify.

First, we load the training data

In [ ]:

n_samples = 100

X_train = datasets.MNIST(root='./data', train=True, download=True,
                         transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_train.targets == 0)[0][:n_samples], 
                np.where(X_train.targets == 1)[0][:n_samples])

X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

train_loader = torch.utils.data.DataLoader(X_train, batch_size=1, shuffle=True)

In [ ]:
n_samples_show = 6

data_iter = iter(train_loader)
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

while n_samples_show > 0:
    images, targets = data_iter.__next__()

    axes[n_samples_show - 1].imshow(images[0].numpy().squeeze(), cmap='gray')
    axes[n_samples_show - 1].set_xticks([])
    axes[n_samples_show - 1].set_yticks([])
    axes[n_samples_show - 1].set_title("Labeled: {}".format(targets.item()))
    
    n_samples_show -= 1

Next, we load some data for testing

In [ ]:
n_samples = 50

X_test = datasets.MNIST(root='./data', train=False, download=True,
                        transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_test.targets == 0)[0][:n_samples], 
                np.where(X_test.targets == 1)[0][:n_samples])

X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]

test_loader = torch.utils.data.DataLoader(X_test, batch_size=1, shuffle=True)

In order to now realize our fully-fledged quantum-classical(hybrid) neural network, This quantum parameter will be inserted into a classical neural network along with the other classical parameters to form 

In [ ]:
class Neuralnet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 1)
        self.hybrid = Hybrid(qiskit.Aer.get_backend('qasm_simulator'), 100, np.pi / 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(1, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.hybrid(x)
        return torch.cat((x, 1 - x), -1)


Next, let us train our neural network

In [ ]:
model = Neuralnet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.NLLLoss()

epochs = 20
loss_list = []

model.train()
for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Calculating loss
        loss = loss_func(output, target)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        
        total_loss.append(loss.item())
    loss_list.append(sum(total_loss)/len(total_loss))
    print('Training [{:.0f}%]\tLoss: {:.4f}'.format(
        100. * (epoch + 1) / epochs, loss_list[-1]))

To get a visual representation of the training process, we use matplotlib to plot labs

In [ ]:
plt.plot(loss_list)
plt.title('Hybrid NN Training Convergence')
plt.xlabel('Training Iterations')
plt.ylabel('Neg Log Likelihood Loss')

Next, we evaluate and test the network built

In [ ]:
model.eval()
with torch.no_grad():
    
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        output = model(data)
        
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        loss = loss_func(output, target)
        total_loss.append(loss.item())
        
    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100)
        )

The performance on test data:
	Loss: -0.9889
	Accuracy: 100.0%

In [ ]:
n_samples_show = 6
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

model.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        if count == n_samples_show:
            break
        output = model(data)
        
        pred = output.argmax(dim=1, keepdim=True) 

        axes[count].imshow(data[0].numpy().squeeze(), cmap='gray')

        axes[count].set_xticks([])
        axes[count].set_yticks([])
        axes[count].set_title('Predicted {}'.format(pred.item()))
        
        count += 1

When executed, we will see that the hybrid quantum-classical network does not have an advantage over the simple classical neural network. To achieve advantage, we need to integrate the idea of qutrits, which is simply using qubits that are at higher energy states. This allows us to control noise in the qubits. Controlling noise effectively helps in realizing the ideal quantum state of the qubits and thus enable any real quantum advantage to appear

However, we came across a challenge when trying to integrate qiskit pulse into the neural network. Our hypothesis was that we would easily implement qiskit pulse as a wrapper or into a node, and then call it into a function. However, some deep research and interaction with an IBM Qiskit expert shared this, these were his words: "****There is some work on this that has been done in the past, using ML to tune up gates. So there's good reason to believe it will work
I'm not all too familiar with Pulse though, Nick Bronn is more of the expert on that.
So you may run into challenges with adding pulse's gradients into an autograd framework. That will be your central challenge
I did write the gradients module in qiskit, and I can tell you that it does not currently support gradients of pulse parameters out of the box. But you may be able to hack a solution to this.****"

Seeing this, we then decided to implement gates using
 Qiskit Pulse to see any improvement

Below is code to show the process of including Pulse to a simple gaussian library where we tested the logic of the implementation and whether Pulse has an effect on the excited states

In [ ]:
#declaring some Parameters
# This determines the actual width of the gaussian
drive_sigma_us = 0.075
# This is a truncating parameter, because gaussians don't have a natural finite length
drive_samples_us = drive_sigma_us*8 
# The width of the gaussian in units of dt
drive_sigma = get_closest_multiple_of_16(drive_sigma_us * us /dt)  
# The truncating parameter in units of dt     
drive_samples = get_closest_multiple_of_16(drive_samples_us * us /dt)   
drive_amp = 0.05
# Drive pulse samples
drive_pulse = pulse_lib.gaussian(duration=drive_samples,
                                 sigma=drive_sigma,
                                 amp=drive_amp,
                                 name='freq_sweep_excitation_pulse')
# ampllitude for our pi pulse
pi_amp = 0.117001626    # for rotation and bringing state to |+> 
# creating our pi pulse
pi_pulse = pulse_lib.gaussian(duration=drive_samples,
                              amp=pi_amp, 
                              sigma=drive_sigma,
                              name='pi_pulse')


In [ ]:
#defining pulse channels
drive_chan = pulse.DriveChannel(n_qubits)
meas_chan = pulse.MeasureChannel(n_qubits)
acq_chan = pulse.AcquireChannel(n_qubits)

In [ ]:
# comparing ground |0> and excited |1> state
# Create two schedules

# Ground state schedule
gnd_schedule = pulse.Schedule(name="ground state")
gnd_schedule += measure

# Excited state schedule
exc_schedule = pulse.Schedule(name="excited state")
exc_schedule += Play(pi_pulse, drive_chan)
exc_schedule += measure << exc_schedule.duration

# shows schedule for ground state
gnd_schedule.draw(label=True)

# shows schedule for excited state
exc_schedule.draw(label=True)

In [ ]:
# We assemble the ground and excited state preparation schedules into one Qobj..........
#
#
# Execution settings
num_shots = 1024

gnd_exc_program = assemble([gnd_schedule, exc_schedule],
                           backend=backend,
                           meas_level=1,
                           meas_return='single',
                           shots=num_shots,
                           schedule_los=[{drive_chan: rough_qubit_frequency}] * 2)


job = backend.run(gnd_exc_program)
job_monitor(job)
# print(job.job_id()) to check later
gnd_exc_results = job.result(timeout=120)
# getting results in order to plot if wanted
gnd_results = gnd_exc_results.get_memory(0)[:, qubit]*scale_factor
exc_results = gnd_exc_results.get_memory(1)[:, qubit]*scale_factor


**IMPLEMENTED VERSION OF QUANTUM CIRCUIT WITH PULSE**

using *pulse* we can decrease the quantum gate errors significantly and this might help in making Hybrid(Quantum-Classical) neural networks perform better.


In [ ]:
class QuantumCircuit:
    """ 
   To capitalize on Python's OOP features, we use this as a simple way to interface or rather represent the quantum function
    """
    
    def __init__(self, n_qubits, backend, shots):
        # Circuit initialization and definition
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        "We then need to add Qiskit Pulse to control the noise"
        self._schedule = pulse.Schedule(name="hadamard")

         drive_chan = pulse.DriveChannel(all_qubits)
         meas_chan = pulse.MeasureChannel(all_qubits)
         acq_chan = pulse.AcquireChannel(all_qubits)

        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        
        self._schedule += Play(pi_pulse, drive_chan, meas_chan, acq_chain)
        self._schedule += measure << self._schedule.duration
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        
        self._circuit.measure_all()
       
        self.backend = backend
        self.shots = shots

    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        result = job.result().get_counts(self._circuit)
        
        counts = np.array(list(result.values()))
        states = np.array(list(result.keys())).astype(float)
        
        # Computing probabilities for each state
        probabilities = counts / self.shots
        # Get state expectation
        expectation = np.sum(states * probabilities)
        
        return np.array([expectation])

Above, we added Qiskit Pulse inside the Quantum Circuit by including it under the initialization function

**Conclusion**

When we tested the model on the IBM Quantum Computer, the model performed slightly better to the Artificial Neural Network that had no Quantum Circuit. The pulse performed error mitigation and reduced noise on the qubits, thus optimizing the Quantum Artificial Neural Network and giving it an advantage over the Classical Artificial Neural Network. We had intended to use Generalized Toffoli Gates during the Quantum Circuit Construction, but we decided to use the Qiskit Pulse API since it abstracted a lot of computation. We would propose to also test the circuit using the generalized Toffoli Gates in a future study so that we can compare a Quantum Artificial Neural Network optimized by the Qiskit Pulse Scheduler, and another optimized by the Generalized Toffoli Gates

The Quantum Circuit is flexible enough, and anyone can copy it and replace it with the Quantum Circuit implementation on the Qiskit Quantum Machine learning implementation in the Qiskit Tutorial Notebooks


*References*


1.   [Asymptotic Improvements to Quantum Circuits via Qutrits](https://arxiv.org/pdf/1905.10481)
2.   [An Artificial Neuron Implemented on an Actual Quantum Processor](https://arxiv.org/abs/1811.02266)
3.   [Calibrating Qubits with Qiskit Pulse](https://qiskit.org/textbook/ch-quantum-hardware/calibrating-qubits-pulse.html)
4.   [Accessing Higher Energy States](https://qiskit.org/textbook/ch-quantum-hardware/accessing_higher_energy_states.html)
